# Tutorial: Async Path Creation with Timeouts

**Category**: ln Utilities
**Difficulty**: Intermediate
**Time**: 15-20 minutes

## Problem Statement

File I/O can hang indefinitely on network file systems or slow disks. Async I/O without timeout protection blocks critical workflows.

**Why This Matters**:
- **Service Reliability**: Prevents hung I/O from blocking async event loops
- **Resource Management**: Timeout protection prevents memory leaks

**What You'll Build**:
Async file creation using `acreate_path` with timeout protection and fallback strategies.

## Prerequisites

**Prior Knowledge**:
- Python async/await fundamentals
- Path manipulation with `pathlib`

**Required Packages**:
```bash
pip install lionherd-core  # >=0.1.0
```

In [ ]:
# Standard library
import asyncio
from pathlib import Path
from dataclasses import dataclass

# lionherd-core
from lionherd_core.ln import acreate_path
from anyio import Path as AsyncPath

## Solution Overview

We'll implement timeout-protected path creation:

1. **Basic Async Creation**: Core API features
2. **Timeout Protection**: Bounded execution times
3. **Fallback Strategies**: Graceful degradation

**Key Components**:
- `acreate_path`: Async path creation with timeout via `move_on_after`
- `AsyncPath` (anyio): Cross-platform async file operations

**Pattern**: Operations complete within timeout, fallback to alternative locations when primary storage is slow.

### Step 1: Basic Async Path Creation

Understand core capabilities: nested directories, timestamps, unique filenames.

**Key Point**: `AsyncPath` provides non-blocking I/O operations.

In [ ]:
async def demo_basic():
    # Simple file
    path1 = await acreate_path(
        directory="/tmp/demo",
        filename="simple",
        extension=".txt"
    )
    print(f"Simple: {path1}")
    
    # With timestamp
    path2 = await acreate_path(
        directory="/tmp/demo",
        filename="logs/app",
        extension=".log",
        timestamp=True,
        timestamp_format="%Y%m%d_%H%M%S"
    )
    print(f"Timestamped: {path2}")
    
    # Unique with hash
    path3 = await acreate_path(
        directory="/tmp/demo",
        filename="unique",
        extension=".json",
        random_hash_digits=8
    )
    print(f"Unique: {path3}")

await demo_basic()

### Step 2: Timeout Protection

Add bounded execution time to prevent indefinite hangs.

**Key Point**: Timeout parameter prevents blocking when file systems become unresponsive.

In [ ]:
@dataclass
class PathConfig:
    default_timeout: float = 2.0
    fast_timeout: float = 0.5

async def create_with_timeout(directory: str, filename: str, config: PathConfig):
    try:
        path = await acreate_path(
            directory=directory,
            filename=filename,
            timeout=config.default_timeout,
            extension=".txt"
        )
        print(f"✓ Created: {path}")
        return path
    except TimeoutError as e:
        print(f"✗ Timeout: {e}")
        raise

config = PathConfig(default_timeout=2.0)
path = await create_with_timeout("/tmp/demo", "protected", config)

### Step 3: Fallback Strategies

Gracefully degrade to alternative locations on timeout.

**Key Point**: Fallbacks enable 99.9% availability even when primary storage has 95% availability.

In [ ]:
async def create_with_fallback(
    directory: str,
    filename: str,
    timeout: float = 2.0,
    alternative_dirs: list[str] = None,
):
    # Try primary
    try:
        return await acreate_path(
            directory=directory,
            filename=filename,
            timeout=timeout,
            extension=".txt"
        )
    except TimeoutError:
        print(f"Primary timed out, trying fallbacks...")
    
    # Try alternatives
    for alt_dir in (alternative_dirs or ["/tmp/fallback"]):
        try:
            path = await acreate_path(
                directory=alt_dir,
                filename=filename,
                timeout=timeout,
                extension=".txt"
            )
            print(f"✓ Fallback: {alt_dir}")
            return path
        except TimeoutError:
            continue
    
    return None

path = await create_with_fallback(
    "/tmp/demo",
    "resilient",
    alternative_dirs=["/tmp/fallback1", "/tmp/fallback2"]
)
if path:
    print(f"Final: {path}")

## Complete Working Example

Production-ready async path creator with comprehensive features.

In [ ]:
"""\nProduction async path creation.\n"""
import asyncio
from dataclasses import dataclass
from lionherd_core.ln import acreate_path

@dataclass
class ResilientConfig:
    default_timeout: float = 2.0
    alternative_dirs: list[str] = None

class ResilientPathCreator:
    def __init__(self, config: ResilientConfig = None):
        self.config = config or ResilientConfig(alternative_dirs=["/tmp/fallback"])
    
    async def create(self, directory: str, filename: str, **kwargs):
        # Try primary
        try:
            return await acreate_path(
                directory=directory,
                filename=filename,
                timeout=self.config.default_timeout,
                **kwargs
            )
        except TimeoutError:
            pass
        
        # Try fallbacks
        for alt_dir in self.config.alternative_dirs:
            try:
                return await acreate_path(
                    directory=alt_dir,
                    filename=filename,
                    timeout=self.config.default_timeout,
                    **kwargs
                )
            except TimeoutError:
                continue
        
        return None

# Usage
creator = ResilientPathCreator()
path = await creator.create(
    "/tmp/demo",
    "production",
    extension=".txt",
    timestamp=True
)
if path:
    print(f"Created: {path}")

## Production Considerations

### Error Handling

```python
def classify_error(error: Exception):
    if isinstance(error, TimeoutError):
        return "timeout"  # Retry with fallback
    elif isinstance(error, PermissionError):
        return "permission"  # Alert ops
    elif isinstance(error, OSError) and "No space" in str(error):
        return "disk_full"  # Trigger cleanup
```

### Performance

- Local SSD: ~0.1-2ms per operation
- Network storage: 5-50ms typical, 100-1000ms under load
- Timeout overhead: <0.1ms

### Testing

```python
async def test_timeout_triggers_fallback():
    config = ResilientConfig(default_timeout=0.001)  # Very tight
    creator = ResilientPathCreator(config)
    result = await creator.create("/mnt/slow_nfs", "test", extension=".txt")
    assert "/tmp" in str(result)  # Used fallback
```

## Variations

### Strict Mode (No Fallback)

```python
async def create_strict(directory, filename, timeout=1.0):
    # Pre-validate directory
    dir_path = AsyncPath(directory)
    if not await dir_path.exists():
        raise ValueError(f"Directory not found: {directory}")
    
    return await acreate_path(
        directory=directory,
        filename=filename,
        timeout=timeout,
        file_exist_ok=False  # Never overwrite
    )
```

### Zero-Timeout (Immediate Fallback)

```python
async def create_fast(directory, filename):
    try:
        return await acreate_path(
            directory=directory,
            filename=filename,
            timeout=0.01  # 10ms max
        )
    except TimeoutError:
        # Immediate local fallback
        return await acreate_path(
            directory="/tmp/urgent",
            filename=filename,
            timeout=0.005
        )
```

## Summary

**What You Accomplished**:
- ✅ Implemented timeout-protected async path creation
- ✅ Built fallback strategies for high availability
- ✅ Created production-ready configuration

**Key Takeaways**:
1. **Timeout protection essential**: Unbounded I/O hangs async apps
2. **Fallbacks enable HA**: Graceful degradation maintains service
3. **Measure everything**: Latency, fallback rate for proactive detection

**When to Use**:
- ✅ Network file systems (NFS, cloud storage)
- ✅ High-throughput services requiring bounded times
- ❌ Simple single-user scripts on local disk

## Related Resources

- [acreate_path API](../../docs/api/ln/utils.md)
- [Trio: Timeouts](https://trio.readthedocs.io/en/stable/reference-core.html#cancellation-and-timeouts)